In [83]:
import lightgbm as lgb
from sklearn import metrics
import category_encoders as ce

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.impute import SimpleImputer

df = pd.read_csv('../data/train.csv',dtype={'tipodepropiedad':'category','ciudad':'category','provincia':'category'})
df['fecha'] = pd.to_datetime(df['fecha'])
df['anio'] = df['fecha'].dt.year
df["mes"] = df['fecha'].dt.month
df["dia"] = df['fecha'].dt.day

df = df.drop(columns=["lat","lng"])

In [84]:
import re
def contiene_frase(texto, frase):
    if re.search(frase, texto,re.IGNORECASE):
        return True
    return False

contiene_frase=np.vectorize(contiene_frase)

In [85]:
def cantidad_de_public(data,colum,palabra):
    return data[contiene_frase(data[colum],palabra)].dia.count()

def serie_que_contiene_palabra(data,colum,palabra):
        return contiene_frase(data[colum],palabra)

# Completo los datos faltantes

In [86]:
imp = SimpleImputer(strategy="constant")
df["provincia"] = imp.fit_transform(df[["provincia"]])
df["ciudad"] = imp.fit_transform(df[["ciudad"]])

In [87]:
df['ciudad-provincia'] = df[['ciudad', 'provincia']].apply(lambda x: ', '.join(x), axis=1)

In [88]:
df = df.reset_index()

# Busco en las descripciones

In [89]:
busqueda = "descripcion"
df1 = df.copy()
df1[busqueda] = df1[busqueda].fillna(value="-")
df1.shape

(240000, 26)

In [90]:
palabra = "excelente"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

68971

In [91]:
palabra = "centro"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

25889

In [92]:
palabra = "patio"
palabra2 = "jardin"
palabra3 = "jardín"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

100824

In [93]:
palabra = "hermosa"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

21198

In [94]:
palabra = "grande"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

11163

In [95]:
palabra = "universidad"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

4221

In [96]:
palabra = "ilum"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)


20875

In [97]:
palabra = "seguridad"
palabra2 = "vigilancia"
palabra3 = "proteccion"
palabra4 = "protecciones"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)|serie_que_contiene_palabra(df1,busqueda,palabra4)
df1[palabra].value_counts()[1]

58005

In [98]:
palabra = "tienda"
palabra2 = "comercial"
palabra3 = "local"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

39496

In [99]:
palabra = "comedor"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

127434

In [100]:
palabra = "lujo"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

13816

In [101]:
palabra = "servicio"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

87104

In [102]:
palabra = "remodelada"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

6379

In [103]:
palabra = "autos"
palabra2 = "auto"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

78290

In [104]:
palabra = "completo"
palabra2 = "completos"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

66453

In [105]:
palabra = "comedor"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

127434

In [106]:
palabra = "cerca de"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

29493

In [107]:
palabra = "con vista"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

9335

# Busco en las direcciones

In [108]:
busqueda = "direccion"

df1[busqueda] = df1[busqueda].fillna(value="-")

In [109]:
palabra = "avenida"
palabra2 = "av"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

20304

In [110]:
df1["largo descripcion"] = df1.descripcion.transform(lambda x: len(x))

# Creo el CSV

In [111]:
df1.to_csv("../data/features.csv", index=False)